In [ ]:
import os
work_dir = "/content/TenserFlow-Tutorials/"
if os.getcwd() != work_dir:
    !git clone http://github.com/Hvass-Labs/TenserFlow-Tutorials.git
os.chdir(work_dir)     
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from mnist import MNIST
from sklearn.metrics import confusion_matrix

In [ ]:
data = load_data(data_dir = "/data/MNIST")
img_size_flat = data.img_size_flat
img_shape = data.img_shape
num_classes = data.num_classes

In [ ]:
# data.y_test[0:5,:] ## One Hot-Encoded Y matrix (5 * 10)
# data.y_test_cls[0:5] ## Y class wise array (1 * 5)

In [ ]:
def plot_images (images, cls_true, cls_pred = None) :
    assert length(images) = length(cls_true) == 9
    fig, axes = plt.subplot(3,3)
    fig.subplots.adjust(hspace = 0.3, wspace = 0.3)
    for i, ax in enumerate(axes.flat):
        ax.imshow(images[i].reshape(img_shape), cmap = 'binary')
        if cls_pred is None :
            xlabel = "True : {0}".format(cls_true[i])
        else
            xlabel = "True : {0} Pred : {1}".format(cls_true[i], cls_pred[i])
    ax.set_xlabel(xlabel)
    ax.set_xticks([])
    ax.set_yticks([])
    plt.show()
# plotting the first 9 images
# images = data.x_test[0:9]
# cls_true = data.y_test_cls[0:9]
# plot_images(images, cls_true = cls_true)

In [ ]:
#Initializing the placeholder variables 
x = tf.placeholder(tf.float32, [None, img_size_flat])
y_true = tf.placeholder(tf.float32, [None, num_classes])
y_true_cls = tf.placeholder(tf.int64, [None])
#initializing the parameters
weights = tf.Variable(tf.zeros[img_size_flat, num_classes])
biases = tf.Variable(tf.zeros[num_classes])

In [ ]:
#Model
logits = tf.matmul(x, wights) + biases
y_pred = tf.nn.softmax(logits)
y_pred_cls = tf.argmax(y_pred, axis = 1)

In [ ]:
#Cost Function
cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits = logits, labels = y_true)
cost = tf.reduce_mean(cross_entropy)

In [ ]:
#Optimizaation Method
optimizer = tf.nn.GradientDescentOptimizer(learning_rate = 0.5).minimize(cost)
correct_prediction = tf.equal(y_pred_cls, y_true_cls)
accuracy = tf.reduce__mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
#Running the program
session = tf.Session()
session.run(tf.global_variables_initializer())
batch_size = 100
def optimize(num_iterations):
    for i in range(num_iterations):
        x_batch, y_true_batch, _ = data.random_batch(batch_size = batch_size)
        feed_dict_train = {x: x_batch, y_true = y_true_batch}
        session.run(optimizer, feed_dict = feed_dict_train)        

In [ ]:
feed_dict_test = {x : data.x_test, y_true : data.y_true, y_true_cls = data.y_test_cls}
#Function to print accuracy
def print_accuracy() :
    acc = session.run(accuracy, feed_dict = feed_dict_test)
    print("The current accuracy is {0: .1%}".format(acc))
#Function to print confusion matrix
def print_confusion_matrix() :
    cls_true = data.y_test_cls
    cls_pred = session.run(y_pred_cls, feed_dict = feed_dict_test)
    cm = confusion_matrix(y_true = cls_true, y_pred = cls_pred)
    print (cm)
    plt.imshow(cm, interpolation = 'nearest', cmap = plt.cm.Blues)
    plt.tight_layout()
    plt.colorbar()
    tick_marks = np.arange(num_classes)
    plt.xticks(tick_marks, range(num_classes))
    plt.yticks(tick_marks, range(num_classes))
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.show()
#Function to print mislabeled examples
def plot_example_errors() :
    correct, class_pred = session.run([correct_prediction, y_pred_cls], feed_dict = feed_dict_test)
    incorrect = (correct == False)
    images = data.x_test[incorrect]
    class_pred = class_pred[incorrect]
    class_true = data.y_test_cls[incorrect]
    plot_images(images[0:9], class_true = class_true[0:9], class_pred = class_pred[0:9])

In [ ]:
#Function to plot the weights of each of the classes
def plot_weights() :
    w = session.run(weigths)
    w_min = np.min(w)
    w_max = np.max(w)
    fig, axes = plt.subplot(3,4)
    fig.subplots_adjust(hspace = 0.3, wspace = 0.3)
    for i, ax in enumerate(axes.flat) :
        if i<10:
            image = w[:, i].reshape(img_shape)
            ax.set_xlabel("Weigths : {0}".format(i))
            ax.imshow(image, vmin = w_min, wmax = w_max, cmap = "seismic")
        ax.set_xticks([])
        ax.set_yticks([])
    plt.show()           

In [ ]:
#Performance without any optimization
print_accuaracy()
plot_example_errors()
plot_weigths()

In [ ]:
#Performance after 1 iteration
optimize(num_iterations = 1)
print_accuracy()
plot_example_errors()
plot_weights()

In [ ]:
#Performance after 100 iterations 
optimize(num_iterations = 100)
print_accuracy()
plot_example_errors()
plot_weights()

In [ ]:
#Performance after 1000 iterations 
optimize(num_iterations = 1000)
print_accuracy()
plot_example_errors()
plot_weights()
print_confusion_matrix()